# Pre-set networks

To speed up experimentation, but above all to ensure better reproducibility of experiments, pre-designed networks have been implemented in sumo-experiments. There are currently three types of pre-designed network :
- A network composed with only one intersection (OneCrossroadNetwork)
- A network composed with two intersections (TwoCrossroadsNetwork)
- A network composed of $n$ crossroads arranged in a square (SquareNetwork)

Each network is represented by a class in the `preset_networks` package. The classes contains methods that generates infrastructures, flows, and all add-ons that can be added to a network.

***Note :** For now, only detectors can be added to a pre-set network. Adding new features is one of our priority.*

Let's see how we create a simulation with a pre-set network. Wi will use the `OneCrossroadNetwork` to do this.

First, we must instanciate the preset network.

In [ ]:
from sumo_experiments.preset_networks import OneCrossroadNetwork

network = OneCrossroadNetwork()

## Pre-set functions

### Infrastructures

Each class contains one and only one method that generates infrastructures (`generate_infrastructures`). It is the function that you must use to create an Experiment. This function return an `InfrastructureBuilder` object, containing all the infrastructures needed for the definition of the network (nodes, edges, connections, traffic light programs). Thus, in this example with the `OneCrossroadNetwork`, the function will return an object containing 5 nodes (including 1 traffic light), 8 edges (4 * 2 directions), and 12 connections.

In [ ]:
type(network.generate_infrastructures())

### Flows

Each class contains three methods to define flows for the network :
- The `generate_flows_only_ahead` method generates vehicles that can't turn in the intersections. They can only go ahead.
- The `generate_flows_all_directions` method generates vehicles that can go into any direction in the intersections (except turning back). The vehicle repartition for all directions is uniform.
- The `generate_flows_with_matrix` method generate variable flows over time that can go anywhere in the intersections. We'll see this in a future section.

Each method returns a `FlowBuilder` object containing all the flows and informations about it (flow entry, exit, density, vehicle type).

In [ ]:
type(network.generate_flows_only_ahead())

Let's do some graphical experiments to show you the difference between the `generate_flows_only_ahead` and the `generate_flows_all_directions`.

In [ ]:
from sumo_experiments import Experiment

exp_only_ahead = Experiment(
    name = 'only_ahead',
    infrastructures = network.generate_infrastructures,
    flows = network.generate_flows_only_ahead
)

exp_all_directions = Experiment(
    name = 'all_directions',
    infrastructures = network.generate_infrastructures,
    flows = network.generate_flows_all_directions
)

exp_only_ahead.run(gui = True)
exp_all_directions.run(gui = True)

exp_only_ahead.clean_files()
exp_all_directions.clean_files()

### Detectors

Optionally, you can add detectors to the intersections of a network to adapt their operation to the traffic. While detectors can be added when generating the pre-set network, adapting the behavior of traffic lights is more complex and will be the subject of another tutorial.

Each pre-set network has two methods to generate detectors :
- `generate_boolean_detectors` generates short-range detectors that check if a car is its range or not.
- `generate_numerical_detectors` generates long-range detectors (all of the edge) that are supposed to count vehicles that are on its range.

The detectors generates data that can be used by the TraCi library to modify the traffic lights behaviour.

To add detectors to a network, just add the `detectors` parameter to an experiment, with the right function to use to generate it.

In [ ]:
exp_detectors = Experiment(
    name = 'detectors',
    infrastructures = network.generate_infrastructures,
    flows = network.generate_flows_all_directions,
    detectors = network.generate_boolean_detectors
)

exp_detectors.run(gui=True)

exp_detectors.clean_files()

***Note :** Some other feautures should be added in the future, like pedestrians, bicycles, etc.*

## Parameters

Until now, we only used pre-set networks with default parameters. It's possible to configure each of the network with some parameters. In order to do so, we must use the `set_parameter` method from the `Experiment` class. Parameters are added into a dictionnary in the `Experiment` object. Then, you can add paremeters in the order you  want. You can also add parameters that can not be use by the pre-set functions. In this case, the `run` method will return a warning when called.

Here is a short example on how to configure the maximum duration of a simulation, with the parameter `simulation_duration`.

In [ ]:
exp_parameters = Experiment(
    name = 'parameters',
    infrastructures = network.generate_infrastructures,
    flows = network.generate_flows_all_directions
)

exp_parameters.set_parameter('simulation_duration', 2000)

exp_parameters.run()

exp_parameters.clean_files()

Each pre-set network has its own parameters, with some in common with other networks. Here is a list for each of the pre-set networks implemented.

### OneCrossroadNetwork

#### Infrastructures

- `lane_length` (int) : The default length for each lane (in meters)
- `north_length` (int) : The north lane length (in meters), override default
- `east_length` (int) : The east lane length (in meters), override default
- `south_length` (int) : The south lane length (in meters), override default
- `west_length` (int) : The west lane length (in meters), override default
- `green_time` (int) : The default green time for each phase (in seconds)
- `green_time_north_south` (int) : The north-south phase green time (in seconds), override default
- `green_time_west_east` (int) : The west-east phase green time (in seconds), override default
- `yellow_time` (int) : The default yellow time for each phase (in seconds)
- `yellow_time_north_south` (int) : The north-south phase yellow time (in seconds), override default
- `yellow_time_west_east` (int) : The west-east phase yellow time (in seconds), override default
- `max_speed` (float) : The max speed on each lane

#### Flows

- `stop_generation_time` (int) : The default simulation step when flows will end
- `stop_generation_time_north` (int) : The simulation step when north flows will end, override default
- `stop_generation_time_east` (int) : The simulation step when east flows will end, override default
- `stop_generation_time_south` (int) : The simulation step when south flows will end, override default
- `stop_generation_time_west` (int) : The simulation step when west flows will end, override default
- `flow_density` (int) : The default flows density (in vehicles/hour)
- `flow_density_north` (int) : The north flows density (in vehicles/hour), override default
- `flow_density_east` (int) : The east flows density (in vehicles/hour), override default
- `flow_density_south` (int) : The south flows density (in vehicles/hour), override default
- `flow_density_west` (int) : The west flows density (in vehicles/hour), override default

##### With matrices

- `coeff_matrix` (numpy.ndarray) : The proportion of vehicles on each route
- `load_vector` (numpy.ndarray) : The vehicle density on the network for each period
- `period_time` (int) : The period duration (in simulation steps)

#### Detectors

- `boolean_detector_length` (float) : The scope size of the detectors

### TwoCrossroadsNetwork

#### Infrastructures

- `lane_length` (int) : The default length for each lane (in meters)
- `north_1_length` (int) : The north lane length (in meters) for intersection 1, override default
- `north_2_length` (int) : The north lane length (in meters) for intersection 2, override default
- `east_length` (int) : The east lane length (in meters), override default
- `south_1_length` (int) : The south lane length (in meters) for intersection 1, override default
- `south_2_length` (int) : The south lane length (in meters) for intersection 2, override default
- `west_length` (int) : The west lane length (in meters), override default
- `center_length` (int): The center lane length (in meters), override default
- `green_time` (int) : The default green time for each phase (in seconds)
- `green_time_north_south_1` (int) : The north-south phase green time (in seconds) for intersection 1, override default
- `green_time_north_south_2` (int) : The north-south phase green time (in seconds) for intersection 2, override default
- `green_time_west_east_1` (int) : The west-east phase green time (in seconds) for intersection 1, override default
- `green_time_west_east_2` (int) : The west-east phase green time (in seconds) for intersection 2, override default
- `yellow_time` (int) : The default yellow time for each phase (in seconds)
- `yellow_time_north_south_1` (int) : The north-south phase yellow time (in seconds) for intersection 1, override default
- `yellow_time_north_south_2` (int) : The north-south phase yellow time (in seconds) for intersection 2, override default
- `yellow_time_west_east_1` (int) : The west-east phase yellow time (in seconds) for intersection 1, override default
- `yellow_time_west_east_2` (int) : The west-east phase yellow time (in seconds) for intersection 2, override default
- `max_speed` (float) : The max speed on each lane

#### Flows

- `stop_generation_time` (int) : The default simulation step when flows will end
- `stop_generation_time_north_1` (int) : The simulation step when north flows will end for intersection 1, override default
- `stop_generation_time_north_2` (int) : The simulation step when north flows will end for intersection 2, override default
- `stop_generation_time_east` (int) : The simulation step when east flows will end, override default
- `stop_generation_time_south_1` (int) : The simulation step when south flows will end for intersection 1, override default
- `stop_generation_time_south_2` (int) : The simulation step when south flows will end for intersection 2, override default
- `stop_generation_time_west` (int) : The simulation step when west flows will end, override default
- `flow_density` (int) : The default flows density (in vehicles/hour)
- `flow_density_north_1` (int) : The north flows density (in vehicles/hour) for intersection 1, override default
- `flow_density_north_2` (int) : The north flows density (in vehicles/hour) for intersection 2, override default
- `flow_density_east` (int) : The east flows density (in vehicles/hour), override default
- `flow_density_south_1` (int) : The south flows density (in vehicles/hour) for intersection 1, override default
- `flow_density_south_2` (int) : The south flows density (in vehicles/hour) for intersection 2, override default
- `flow_density_west` (int) : The west flows density (in vehicles/hour), override default

##### With matrices

- `coeff_matrix` (numpy.ndarray) : The proportion of vehicles on each route
- `load_vector` (numpy.ndarray) : The vehicle density on the network for each period
- `period_time` (int) : The period duration (in simulation steps)

#### Detectors

- `boolean_detector_length` (float) : The scope size of the detectors

### SquareNetwork

#### Infrastructures

- `lane_length` (int) : The length of all edges (in meters)
- `green_time` (int) : The default green time for each phase (in seconds)
- `yellow_time` (int) : The default yellow time for each phase (in seconds)lt
- `max_speed` (float) : The max speed on each lane
- `square_side_length` (int) : The number of intersections that compose a side of the square

##### For random square network

- `minimum_edge_length` (int) : The minimum length of an edge (in meters)
- `maximum_edge_length` (int) : The maximum length of an edge (in meters)

#### Flows

- `stop_generation_time` (int) : The default simulation step when flows will end
- `flow_density` (int) : The default flows density (in vehicles/hour)

##### With matrices

- `coeff_matrix` (numpy.ndarray) : The proportion of vehicles on each route
- `load_vector` (numpy.ndarray) : The vehicle density on the network for each period
- `period_time` (int) : The period duration (in simulation steps)

#### Detectors

- `boolean_detector_length` (float) : The scope size of the detectors

In the next tutorial, we'll see how to create experiments with variable flows, using the `generate_flows_with_matrix` from a pre-set network object.